# Team's notebook for final model

This was our team's final notebook used to generate our model used in the submission.  

**Note: Set your google colab runtime type to GPU(You can thank me later :p)**  

If you are looking for the benchmark models notebook or the object detection notebook, please look for those notebooks respectively.

Note: I have covered most of what is happening here in the other notebook, hence why the short descriptions. If you want a more detailed walkthrough, I suggest you look through the benchmark models notebook.

---

# Setting up the notebook

Same as the other notebooks, we will setting up the main directories and importing the libraries and modules used.


In order to mount our google drive folder to our colab notebook


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Importing main libraries and modules


In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, Activation, GlobalAveragePooling2D, Dense, Dropout, MaxPooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.applications.vgg16 import VGG16 

Using TensorFlow backend.


Setting up base directories

In [3]:
import os

base_dir = 'gdrive/My Drive/Brainhack_TIL_2019_Final/Dataset'

# Directory to our training data
train_folder = os.path.join(base_dir, 'train')

# Directory to our validation data
val_folder = os.path.join(base_dir, 'val')


# List folders and number of files
print("Directory, Number of files")
for root, subdirs, files in os.walk(base_dir):
    print(root, len(files))
    

from keras.preprocessing.image import ImageDataGenerator

# Batch size
bs = 16

# All images will be resized to this value
image_size = (224, 224)

# All images will be rescaled by 1./255. We apply data augmentation here.
train_datagen = ImageDataGenerator(rescale=1./255,
                                   brightness_range= [0.5,1.5],
                                   horizontal_flip=True,
                                  rotation_range=40,
                                  width_shift_range=(-100,100),
                                  height_shift_range=(-100,100))
val_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
print("Preparing generator for train dataset")
train_generator = train_datagen.flow_from_directory(
    directory= train_folder, # This is the source directory for training images 
    target_size=image_size, # All images will be resized to value set in image_size
    batch_size=bs,
    class_mode='categorical')

# Flow validation images in batches of 32 using val_datagen generator
print("Preparing generator for validation dataset")
val_generator = val_datagen.flow_from_directory(
    directory= val_folder, 
    target_size=image_size,
    batch_size=bs,
    class_mode='categorical')

Directory, Number of files
gdrive/My Drive/Brainhack_TIL_2019_Final/Dataset 0
gdrive/My Drive/Brainhack_TIL_2019_Final/Dataset/val 0
gdrive/My Drive/Brainhack_TIL_2019_Final/Dataset/val/EaglePose 27
gdrive/My Drive/Brainhack_TIL_2019_Final/Dataset/val/ChestBump 17
gdrive/My Drive/Brainhack_TIL_2019_Final/Dataset/val/ChildPose 22
gdrive/My Drive/Brainhack_TIL_2019_Final/Dataset/val/Dabbing 23
gdrive/My Drive/Brainhack_TIL_2019_Final/Dataset/val/ChairPose 23
gdrive/My Drive/Brainhack_TIL_2019_Final/Dataset/val/Spiderman 26
gdrive/My Drive/Brainhack_TIL_2019_Final/Dataset/val/HandGun 26
gdrive/My Drive/Brainhack_TIL_2019_Final/Dataset/val/WarriorPose 22
gdrive/My Drive/Brainhack_TIL_2019_Final/Dataset/val/KungfuSalute 25
gdrive/My Drive/Brainhack_TIL_2019_Final/Dataset/val/Salute 24
gdrive/My Drive/Brainhack_TIL_2019_Final/Dataset/val/KoreanHeart 22
gdrive/My Drive/Brainhack_TIL_2019_Final/Dataset/val/KungfuCrane 21
gdrive/My Drive/Brainhack_TIL_2019_Final/Dataset/val/HulkSmash 24
gdrive/

# Constructing the model's architecture


In [4]:
import keras
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
import keras.backend as K
from keras.applications.resnet50 import ResNet50 
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
import cv2, numpy as np

# Here we specify the input shape of our data 
# This should match the size of images ('image_size') along with the number of channels (3)
input_shape = (224, 224,3)
input_img = Input(shape=input_shape)
# Define the number of classes
num_classes = 11

# Defining a baseline model. Here we use the [keras functional api](https://keras.io/getting-started/functional-api-guide) to build the model. 
# TODO: explore different architectures and training schemes
x=ZeroPadding2D((3,3))(input_img)
x=keras.applications.vgg16.VGG16(include_top=False, classes=15, input_shape = (224, 224,3), pooling='avg')(x)


x=Dropout(0.5)(x)
x=Dense(128, activation='relu')(x)
x=Dropout(0.5)(x)
predictions = Dense(15, activation='softmax')(x)

model = Model(inputs=input_img, outputs=predictions)

W0621 03:38:11.447985 139723782072192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0621 03:38:11.495411 139723782072192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0621 03:38:11.545013 139723782072192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0621 03:38:11.595243 139723782072192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



58892288/58889256 [==============================] - 2s 0us/step


W0621 03:38:14.310295 139723782072192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0621 03:38:14.311811 139723782072192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0621 03:38:14.935265 139723782072192 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
from keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(nesterov=True),
              metrics=['accuracy'])
from keras.callbacks import ModelCheckpoint

bestValidationCheckpointer = ModelCheckpoint('saved_model.hdf5', monitor='val_acc', save_best_only=True, verbose=1)

W0621 03:38:16.167474 139723782072192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



# Training the model

In [6]:
history = model.fit_generator(
        train_generator, # train generator has 973 train images
        steps_per_epoch=train_generator.samples // bs + 1,
        epochs=50,
        validation_data=val_generator, # validation generator has 253 validation images
        validation_steps=val_generator.samples // bs + 1,
        callbacks=[bestValidationCheckpointer]
)

W0621 03:38:18.347114 139723782072192 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
18/83 [=====>........................] - ETA: 34:38 - loss: nan - acc: 0.0868

KeyboardInterrupt: ignored

# Submitting the model

Loading the saved model into the notebook

In [0]:
from keras.models import load_model

model_path = 'gdrive/My Drive/DSTA/bestsofar.hdf5'
model = load_model( model_path )

From here on, the code was provided by DSTA to submit our model to their server's, as such I cannot comment much on the code except that it works and do not touch it unless you know what you are doing.

In [0]:
!pip install --force-reinstall --user --no-warn-script-location -q https://ai-camp.s3-us-west-2.amazonaws.com/AiCampEval-1.7-py3-none-any.whl

In [0]:
from AiCampEval import eval_submit

In [0]:
import os
from keras.preprocessing.image import ImageDataGenerator

base_dir = 'gdrive/My Drive/DSTA'
train_folder = os.path.join(base_dir, 'train')

# Batch size
bs = 32

# All images will be resized to this value
image_size = (224, 224)

train_datagen = ImageDataGenerator(rescale=1./255,
                                   brightness_range= [0.5,1.5],
                                   horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
    directory= train_folder, # This is the source directory for training images 
    target_size=image_size, # All images will be resized to value set in image_size
    batch_size=bs,
    class_mode='categorical')

print(train_generator.class_indices)

In [0]:
import numpy as np
from PIL import Image

'''
This is a helper function to preprocess the input before sending it through my network. 
It performs the following steps:
1. Resizes each image to (224,224,3), which are the image dimensions my model was trained on.
2. Normalizes the range of each pixel's value from [0-255] to [0-1].
3. Reshapes each array such that it has a batch dimension: from  (224,224,3) -> (1,224,224,3), 
because my model.predict function expects a batch dimension.

Implement your own preprocessing function according to your model's needs.
''' 
def preprocess_arrays(list_of_np_arrays):
    target_size = image_size
    pil_images = [ Image.fromarray(arr.astype('uint8'), 'RGB') for arr in list_of_np_arrays ]
    resized_pil_images = [img.resize( target_size, Image.NEAREST ) for img in pil_images]
    resized_images = [np.array( img ) for img in resized_pil_images]
    preprocessed_images = [np.expand_dims(x, axis=0) / 255. for x in resized_images]
    return preprocessed_images

'''
This function accepts a list of numpy array images as input and outputs a list of prediction strings 
corresponding to each image in the input list.
IMPORTANT: you have to implement this function.
''' 
def evaluate_images(list_of_np_arrays):
    # Swap the key and value in the class_indices
    label_dict = dict((v,k) for k,v in (train_generator.class_indices).items())
    # Run through the list_of_np_arrays to perform any preprocessing you need
    preprocessed_imgs = preprocess_arrays( list_of_np_arrays )
    # Get a list of softmax_vectors by passing in the preprocessed_imgs to model.predict()
    softmax_vectors = [ model.predict( x )[0] for x in preprocessed_imgs ]
    # Get the predicted_class_indices by running each of the softmax_vectors through np.argmax()
    predicted_class_indices = [ np.argmax( x ) for x in softmax_vectors ]
    # Convert the list of class_indices to a list of predictions (in str)
    # predictions is a list of labels: ['KoreanHeart', 'KoreanHeart','ChairPose',.......]
    predictions = [ label_dict[k] for k in predicted_class_indices ]
    return predictions

In [0]:
## CHANGE HERE
TEAM_ID = ".NEET"
SUBMISSION_TYPE = "thefinaltest_4_88888"

eval_submit(evaluate_images, SUBMISSION_TYPE, TEAM_ID )